#### Note: I excluded the script that gathered tweets from this because it took about 3 hours to get tweets and then I ended up having to wrangle it manually

In [8]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
import pandas as pd
from string import punctuation
nlp = spacy.load('en')

In [9]:
wordlist = []

In [10]:
data = pd.read_csv('gatheredTweets.csv', sep=',', encoding='utf-8')

In [11]:
data.head()

,PublishDate,Tweets
0,4/20/20 21:04,"Just across Dutch border, study following 1,00..."
1,4/20/20 21:00,North Carolina's top health official said that...
2,4/20/20 21:00,Millions of Americans & their children were le...
3,4/20/20 21:00,All the experts say we need to greatly expand ...
4,4/20/20 21:00,WATCH: The 5 teenage girls on this robotics te...


In [12]:
# Set the columnnames for the final dataframe
finalcolumnnames = ['PublishDate', 'Tweets']
df = pd.DataFrame(data)
finaldf = pd.DataFrame(columns=finalcolumnnames)

finalworddf = pd.DataFrame(columns=["Words", "PublishDate"])

In [14]:
# This is the same framework for cleaning that I created for the articles
#for i in range(len(100))
for i in range(len(data)):
    tweetVar = df.loc[i, "Tweets"]
    publishdatevar = df.loc[i, "PublishDate"]

    dataAfterLemmaFilter = []
    dataAfterPronounFilter = []
    dataAfterStopwords = []
    dataAfterPunctuations = []
    dataAfterNounFilter = []

    doc = nlp(str(tweetVar))

    for token in doc:
        dataAfterLemmaFilter.append(token.lemma_)

    for token in dataAfterLemmaFilter:
        if token != "-PRON-":
            dataAfterPronounFilter.append(token.lower().strip())

    stopwords = list(STOP_WORDS)

    for token in dataAfterPronounFilter:
        if token != stopwords:
            dataAfterStopwords.append(token)

    custom_remove_list = ['our', 'live', 'coverage', 'of', 'the', 'has', 'moved', 'here', '\\n']

    dataAfterCustomStopWords = []
    for word in dataAfterStopwords:
        if word not in custom_remove_list:
            dataAfterCustomStopWords.append(word)

    dataAfterRemoveCoverage = []
    for c in dataAfterCustomStopWords:
        if c != 'coverage':
            dataAfterRemoveCoverage.append(c)

    punctuations = punctuation

    for token in dataAfterRemoveCoverage:
        if token not in punctuations:
            dataAfterPunctuations.append(token)

    for value in dataAfterPunctuations:
        td = nlp(value)
        for t in td:
            if t.pos_ == 'NOUN':
                dataAfterNounFilter.append(t)

    dataAfterNounFilterStringFormatting = []

    for nlpObject in dataAfterNounFilter:
        nlpObjectIntoString=str(nlpObject)
        dataAfterNounFilterStringFormatting.append(nlpObjectIntoString)

    cleaned_data_list = dataAfterNounFilterStringFormatting

    df2 = pd.DataFrame({"Tweets": [tweetVar],
                        "PublishDate": [publishdatevar],
                        "CleanedTweet": [cleaned_data_list]})

    finaldf = finaldf.append(df2, ignore_index=True)

In [15]:
finaldf.head()

,PublishDate,Tweets,CleanedTweet
0,4/20/20 21:04,"Just across Dutch border, study following 1,00...","[border, study, people, plan, deal, virus]"
1,4/20/20 21:00,North Carolina's top health official said that...,"[health, result, death, state, month, flu]"
2,4/20/20 21:00,Millions of Americans & their children were le...,"[child, package, immigration, technicality, us..."
3,4/20/20 21:00,All the experts say we need to greatly expand ...,"[expert, testing, capacity, promise, test, end..."
4,4/20/20 21:00,WATCH: The 5 teenage girls on this robotics te...,"[girl, team, use, part, ventilator, device, life]"


In [16]:
finaldf.to_csv('cleanedTweetText.csv', sep=',', encoding='utf-8')